# Arabic Poetry Category Classfication 

## Done By Hamad S. Alassafi


### The notebook will be divided into 8 section

- Business Understanding

- Data Understanding

- Data Preprocessing

- Exploratory Data Analysis

- Model Selection & Developemnt

- Model Evaluation

- Conclusion & Results

- Model Exporting & Notebook Containerization

# Business Understanding

**We have to build a classifier that takes poem of and classify it category as follows:**

- العراق
- سوريا
- مصر تونس
- السعودية
- لبنان
- فلسطين
- اليمن
- العصر الجاهلي 
- العصر الأندلسي
- السودان 
- البحرين
- العصر الإسلامي
- العصر العباسي 
- ليبيا
- عمان
- المغرب
- الجزائر 
- الإمارات 
- الأردن
- الكويت
- قطر
- موريتانيا
- شعراء العراق والشام 
- أفغانستان
- إيران

In [108]:
import pandas as pd
import re
import pickle


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
stemmer = ISRIStemmer()


nltk.download('punkt')
arabic_stopwords = set(stopwords.words('arabic'))

from PIL import Image
import cv2



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer





pd.set_option('display.max_colwidth', None)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hamadalassafi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hamadalassafi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Data Understanding

**We should have inital understanding about our data in this section:**

1. How can we extract the data (e.g. APIs Flat files etc...)
2. Data Profiling
    - Number of poems (rows)
    - Number of columns
    - Name of columns
    - data types

### Data Extraction

In [64]:
df = pd.read_csv("all_poems.csv")

### Data Profiling
    - Number of poems (rows)
    - Number of columns
    - Name of columns
    - Data types

In [65]:
print("Number of poems in our dataset is: ", df.shape[0])

Number of poems in our dataset is:  58021


In [66]:
print("Number of columns in our dataset is: ", df.shape[1])

Number of columns in our dataset is:  9


In [67]:
print("The nine columns we have in our dataset:\n ", df.columns)

The nine columns we have in our dataset:
  Index(['poem_id', 'poem_link', 'poem_style', 'poem_text', 'poem_title',
       'poet_cat', 'poet_id', 'poet_link', 'poet_name'],
      dtype='object')


In [68]:
#Data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58021 entries, 0 to 58020
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   poem_id     58021 non-null  int64 
 1   poem_link   58021 non-null  object
 2   poem_style  58021 non-null  object
 3   poem_text   58020 non-null  object
 4   poem_title  58021 non-null  object
 5   poet_cat    58021 non-null  object
 6   poet_id     58021 non-null  int64 
 7   poet_link   58021 non-null  object
 8   poet_name   58021 non-null  object
dtypes: int64(2), object(7)
memory usage: 4.0+ MB


In [69]:
df.head()

,poem_id,poem_link,poem_style,poem_text,poem_title,poet_cat,poet_id,poet_link,poet_name
0,21,http://www.adab.com/modules.php?name=Sh3er&doWhat=shqas&qid=21&r=&rc=0,فصحى,"عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي عنهما القمر عيناك حين تبسمان تورق الكروم وترقص الاضواء كالاقمار في نهر يرجه المجذاف وهنا ساعة السحر كانما تنبض في غوريهما النجوم وتغرقان في ضباب من اسي شفيف كالبحر سرح اليدين فوقه المساء دفء الشتاء فيه وارتعاشة الخريف والموت والميلاد والظلام والضياء فتستفيق ملء روحي رعشة البكاء ونشوة وحشية تعانق السماء كنشوة الطفل اذا خاف من القمر كان اقواس السحاب تشرب الغيوم وقطرة فقطرة تذوب في المطر وكركر الاطفال في عراءش الكروم ودغدغت صمت العصافير علي الشجر انشودة المطر مطر مطر مطر تثاءب المساء والغيوم ما تزال تسح ما تسح من دموعها الثقال كان طفلا بات يهذي قبل ان ينام بان امه – التي افاق منذ عام فلم يجدها ثم حين لج في السءال قالوا له ""بعد غد تعود "" لا بد ان تعود وان تهامس الرفاق انها هناك في جانب التل تنام نومة الحود تسف من ترابها وتشرب المطر كان صيادا حزينا يجمع الشباك ويلعن المياه والقدر وينثر الغناء حيث يافل القمر مطر مطر اتعلمين اي حزن يبعث المطر وكيف تنشج المزاريب اذا انهمر وكيف يشعر الوحيد فيه بالضياع بلا انتهاء – كالدم المراق كالجياع كالحب كالاطفال كالموتي – هو المطر ومقلتاك بي تطيفان مع المطر وعبر امواج الخليج تمسح البروق سواحل العراق بالنجوم والمحار كانها تهم بالشروق فيسحب اليل عليها من دم دثار اصيح بالخليج "" يا خليج يا واهب الءلء والمحار والردي "" فيرجع الصدي كانه النشيج "" يا خليج يا واهب المحار والردي "" اكاد اسمع العراق يذخر الرعود ويخزن البروق في السهول والجبال حتي اذا ما فض عنها ختمها الرجال لم ترك الرياح من ثمود في الواد من اثر اكاد اسمع النخيل يشرب المطر واسمع القري تءن والمهاجرين يصارعون بالمجاذيف وبالقلوع عواصف الخليج والرعود منشدين "" مطر مطر مطر وفي العراق جوع وينثر الغلال فيه موسم الحصاد لتشبع الغربان والجراد وتطحن الشوان والحجر رحي تدور في الحقول حولها بشر مطر مطر مطر وكم ذرفنا ليلة الرحيل من دموع ثم اعتلنا – خوف ان نلام – بالمطر مطر مطر ومنذ ان كنا صغارا كانت السماء تغيم في الشتاء ويهطل المطر وكل عام – حين يعشب الثري – نجوع ما مر عام والعراق ليس فيه جوع مطر مطر مطر في كل قطرة من المطر حمراء او صفراء من اجنة الزهر وكل دمعة من الجياع والعراة وكل قطرة تراق من دم العبيد فهي ابتسام في انتظار مبسم جديد او حلمة توردت علي فم الوليد في عالم الغد الفتي واهب الحياة مطر مطر مطر سيعشب العراق بالمطر "" اصيح بالخليج "" يا خليج يا واهب الءلء والمحار والردي "" فيرجع الصدي كانه النشيج "" يا خليج يا واهب المحار والردي "" وينثر الخليج من هباته الكثار علي الرمال رغوه الاجاج والمحار وما تبقي من عظام باءس غريق من المهاجرين ظل يشرب الردي من لجة الخليج والقرار وفي العراق الف افعي تشرب الرحيق من زهرة يربها الفرات بالندي واسمع الصدي يرن في الخليج "" مطر مطر مطر في كل قطرة من المطر حمراء او صفراء من اجنة الزهر وكل دمعة من الجياع والعراة وكل قطرة تراق من دم العبيد فهي ابتسام في انتظار مبسم جديد او حلمة توردت علي فم الوليد في عالم الغد الفتي واهب الحياة "" ويهطل المطر",أنشودة المطر,العراق,2,http://www.adab.com/modules.php?name=Sh3er&doWhat=lsq&shid=2&start=0,بدر شاكر السياب
1,65546,http://www.adab.com/modules.php?name=Sh3er&doWhat=shqas&qid=65546&r=&rc=1,فصحى,"انا لا ازال و في يدي قدحي ياليل اين تفرق الشرب ما زلت اشربها و اشربها حتي ترنح افقك الرحب الشرق عفر بالضباب فما يبدو فاين سناك يا غرب ما لنجوم غرقن من سام في ضوءهن و كادت الشهب انا لا ازال و في يدي قدحي ياليل اين تفرق الشرب ****** الحان بالشهوات مصطخب حتي يكاد بهن ينهار و كان مصاحبيه من ضرج كفان مدهما لي العار كفان بل ثغران قد صبغا بدم تدفق منه تيار كاسان ملءهما طلي عصرت من مهجتين رماهما الحب او مخلبان عليهما مزق حمراء تزعم انها قلب ****** الخمر جمعت الدهور , ومافيهن بين جوانب الحان ياويحها اسكرت ام سكرتام نحن في السكرات سيان رمت العوالم والدهور علي ثغري وفوق يدي واجفاني كفي تمد فما تناولني كاسا لعيني خمرها نهب واصافح الدنيا فياعجبا البعد لان واعرض القرب ****** يا ليل اين تطوف بي قدمي في اي منعرج من الظلم تلك السبيل اكاد اعرفها بالامس خاصر طيفها حلمي هي غمد خنجرك الرهيب و قد جردته و مسحت عنه دمي تلك السبيل علي جوانبها تمزق الخطوات او تكبو تثاءب الاجساد جاءعة فيها كما يتثاءب الذءب حسناء يلهب عريها ظماي فاكاد اشرب ذلك العريا و اكاد احطمه فتحطمني عينان جاءعتان كالدنيا غرست يد ال

In [70]:
df.drop(columns=["poem_link", "poem_style", "poem_title", "poet_id", "poet_link", 'poet_name'], inplace = True)

**In the above step, I have removed the unneeded columns to solve the probelm, I might revisit this part based on the later steps**

# Data Preprocessing

- Checking for the Data Quality Six Dimensions

- Completeness. 

- Uniqueness.

- Timeliness. ==> Does not applicable because we do not deal with data that has timing element

- Validity. ==> Does not applicable because we do not deal with data that has issues in terms of validity

- Accuracy.

- Consistency. ==> Does not applicable because we do not deal with data that have different set of representation in terms of unit (e.g. kg g and etc...)


**Since we are dealing with text data so I have to add extra steps of preprocessing:**

- Remove Punctuation and Special Characters

- Normalize Arabic Text

- Remove Diacritics (Tashkeel)

- Remove Stop Words

- Tokenization

- Stemming or Lemmatization

In [71]:
# Completeness

df.isnull().sum()

poem_id      0
poem_text    1
poet_cat     0
dtype: int64

In [72]:
df.dropna(inplace=True)

**We have a single poem text missing, so we will delete it, since it does not effect the size of the data, and since the text is main feature in our solution**

In [73]:
# Uniqueness

df.duplicated(subset = ["poem_id"]).sum()

0

**We do not have any duplicated rows in the dataset**

In [74]:
# Accuracy

def contains_arabic(text):
    arabic_pattern = re.compile(r'[\u0600-\u06FF]')
    return bool(arabic_pattern.search(text))

df['Is_Arabic'] = df['poem_text'].apply(contains_arabic)


In [75]:
df[df["Is_Arabic"] == False]

,poem_id,poem_text,poet_cat,Is_Arabic
15448,54382,,العصر العباسي,False


In [76]:
df = df[df["Is_Arabic"] == True]

**We have a single record that contains non-Arabic text**

In [77]:
def removePunctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [78]:
def normalizeArabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    return text

In [79]:
def removeDiacritics(text):
    arabic_diacritics = re.compile(r' ّ | َ | ً | ُ | ٌ | ِ | ٍ | ْ ', re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)

In [80]:
def removeStopWords(text):
    words = text.split()
    return ' '.join([word for word in words if word not in arabic_stopwords])

In [81]:

def tokenize(text):
    return word_tokenize(text)

In [82]:
def stem(text):
    words = word_tokenize(text)
    return ' '.join([stemmer.stem(word) for word in words])

In [83]:
def textPreprocessing(text):
    text = removePunctuation(text)
    text = normalizeArabic(text)
    text = removeDiacritics(text)
    text = removeStopWords(text)
    text = stem(text)
    text = tokenize(text)
    return text


# Exploratory Data Analysis

- Number of poems per category


In [84]:
df["poet_cat"].value_counts()

العصر العباسي          19416
العصر الأندلسي          6296
سوريا                   5036
لبنان                   4800
مصر                     4793
العراق                  4252
العصر الإسلامي          3700
السعودية                1931
فلسطين                  1713
العصر الجاهلي           1461
اليمن                    700
عمان                     619
الإمارات                 563
البحرين                  554
الجزائر                  405
السودان                  403
تونس                     351
الأردن                   265
ليبيا                    240
المغرب                   202
إيران                     90
أفغانستان                 70
شعراء العراق والشام       68
الكويت                    54
قطر                       22
موريتانيا                 15
Name: poet_cat, dtype: int64

**As we noticed the variance of poems categories is large, I will categories that contain at least 1000 poem**

In [85]:
# Step 1: Calculate the value counts for the column
value_counts = df['poet_cat'].value_counts()

# Step 2: Filter to keep only values with counts >= 100
filtered_values = value_counts[value_counts >= 1000].index

# Step 3: Filter the DataFrame to keep only rows with these values
filtered_df = df[df['poet_cat'].isin(filtered_values)]

In [86]:
filtered_df["poet_cat"].value_counts()

العصر العباسي     19416
العصر الأندلسي     6296
سوريا              5036
لبنان              4800
مصر                4793
العراق             4252
العصر الإسلامي     3700
السعودية           1931
فلسطين             1713
العصر الجاهلي      1461
Name: poet_cat, dtype: int64

**Now we have 10 categories out of 25, after we removed the categories that has less than 1000 poem**

# Model Selection & Developemnt

- We will split the training set into (training and testing sets)
- We will select multiple well-sutied model for this dataset
- Set the parameters and settings
- Train them

**Since we are dealing with images the type of models is going to be CNN (Convolution Neural Network) and we are going to start with a simple model**

In [88]:
X = filtered_df["poem_text"]
y = filtered_df["poet_cat"]

In [89]:
X.shape, y.shape

((53398,), (53398,))

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

In [97]:
pipeline = Pipeline([
                     ('vect', CountVectorizer(tokenizer=textPreprocessing)),
                     ('rfc', RandomForestClassifier(n_estimators=300))
])

pipeline.fit(X_train, y_train)

/Users/hamadalassafi/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function textPreprocessing at 0x329372b00>)),
                ('rfc', RandomForestClassifier(n_estimators=300))])

Training Set Score : 0.9999464925892236
Testing Set Score : 0.4682896379525593


**The main steps is below:**

- We have build a pipeline that takes two things: 

    - Transfomer --CountVectorizer-- We used it to transform the text into numbers after applying the preprocessing steps into the text itself

    - Estimators --RandomForestClassifier-- which classify the text into different categories, and the best thing about Random Forests it's ability to extract results even if the classes is hard to predict, because it uses multiple learners (Decision Trees), and I have increased the number of trees from default (100), to (300)

    - I will use Grid Search, to find the most optimal hyperparamaters

In [100]:
parms = {'rfc__n_estimators':[100,200, 300], 'rfc__max_depth': [3,5,8], "rfc__min_samples_split": [2, 4, 8]}


cv = GridSearchCV(pipeline, parms)

cv.fit(X_train, y_train)

/Users/hamadalassafi/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/hamadalassafi/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/hamadalassafi/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/hamadalassafi/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Users/hamadalassafi/anaconda3/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 't

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function textPreprocessing at 0x329372b00>)),
                                       ('rfc',
                                        RandomForestClassifier(n_estimators=300))]),
             param_grid={'rfc__max_depth': [3, 5, 8],
                         'rfc__min_samples_split': [2, 4, 8],
                         'rfc__n_estimators': [100, 200, 300]})

# Model Evaluation

- Compare two models in terms of accuracy

In [101]:
pred = pipeline.predict(X_test)

print('Training Set Score :' , pipeline.score(X_train,y_train))



print('Testing Set Score :' , accuracy_score(y_test,pred))

Training Set Score : 0.9999464925892236
Testing Set Score : 0.4682896379525593


**Based on the test results, the model clearly suffer from overfitting, it might be because of number of estimators is too much**

In [102]:
pred = cv.predict(X_test)

print('Training Set Score :' , cv.score(X_train,y_train))



print('Testing Set Score :' , accuracy_score(y_test,pred))


Training Set Score : 0.3761035903472631
Testing Set Score : 0.3702247191011236


In [103]:
print(cv.best_estimator_)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function textPreprocessing at 0x329372b00>)),
                ('rfc',
                 RandomForestClassifier(max_depth=8, min_samples_split=8))])


**The increase of the model complexity does not increased the model performance**

# Conclusion & Results


- We started understanding the problem which is poem category classfiaction.

- Then we inspected the poems at a high level and discovered that we have 58021 poem.
- We applied multiple data preprocessing techniques to the text to make them readable by the Machine Learning model.

- We understood the data more and the underlying distribution and almost 33% of the poems have category of (العصر العباسي).

- We have developed two Random Forests Classifiers.

- We fit our data to the model, and discovered overfitting issues.

- We evaluated them and discovered that the first model have better overall performance in the test set with a 9% difference.

- With extra steps in the Exploratory Data Analysis, and the modeling part I think we could have better results. But due to time limitions, I do not have the time to do so


# Model Exporting & Notebook Containerization

In [106]:
with open('modelOne.pkl', 'wb') as file:
     pickle.dump(pipeline, file)

In [107]:
with open('modelTwo.pkl', 'wb') as file:
     pickle.dump(cv, file)

**Note: These two extract files**

- modelOne.pkl
- modelTwo.pkl

**AAre not included in the folder, since it has large size, and it cannot be uploaded to GitHub Repository**